In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

def crawl_item(url_input):
  try:
    url = 'https://mediamart.vn' + url_input
    headers = {
      'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
      'X-Requested-With': 'XMLHttpRequest',
    }    
    r = requests.get(url, headers=headers)
    data = {}
    soup = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')
    data["name"] = soup.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
    x=0
    key=''
    for tr in soup.find_all('td'):
      if(len(tr.get_text()) > 0):
        if(x%2)==0:
          key=tr.get_text()
        else:
          data[key]=tr.get_text()
        x=x+1
    data["price"] = soup.find("div", class_="pdetail-price-box").text
    time.sleep(10)
    return data
  except:
    pass

def crawl_mediaMart():
  data=[]
  df = pd.read_csv('mediamart.csv') #duong dan file csv
  url = 'https://mediamart.vn/smartphones'
  headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'X-Requested-With': 'XMLHttpRequest',
  }    
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.content, 'html.parser')
  for div in soup.find_all("div", class_="col-6 col-md-3 col-lg-3"):
    data.append(crawl_item(div.find("a", href=True)['href']))
  pd.concat([pd.DataFrame(data), df], ignore_index=True)
  return df

In [ ]:
!pip install beautifulsoup4

In [ ]:
url = 'https://mediamart.vn/smartphones/apple-iphone-11-64gb-white'
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'X-Requested-With': 'XMLHttpRequest',
}    
r = requests.get(url, headers=headers)
data = {}
soup = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')

data["name"] = soup.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
x=0
key=''
for tr in soup.find_all('td'):
  if(len(tr.get_text()) > 0):
    if(x%2)==0:
      key=tr.get_text()
    else:
      data[key]=tr.get_text()
    x=x+1
data["price"] = soup.find("div", class_="pdetail-price-box").text
print(data)
jsonString = json.dumps(data, ensure_ascii=False)
jsonFile = open("/content/drive/MyDrive/Colab Notebooks/Media_mart/1.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
f = open("./Media_Mart.txt", "r")
soup = BeautifulSoup(f.read(), 'html.parser')

In [ ]:
soup

In [ ]:
images_link = []
for img in soup.find_all("div", class_ = "card-img-top"):
  images_link.append(img.find("img")['src'])

In [ ]:
images_link = images_link[:147]

In [ ]:
len(soup.find_all("div", class_="col-6 col-md-3 col-lg-3"))

In [ ]:
import time
k = 0
with open("Media_Mart.txt", mode = "w", encoding = "utf8") as f:
  for div in soup.find_all("div", class_="col-6 col-md-3 col-lg-3"):
    try:
      url = 'https://mediamart.vn' + div.find("a", href=True)['href']
      headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
        'X-Requested-With': 'XMLHttpRequest',
      }    
      r = requests.get(url, headers=headers)
      data = {}
      detail = BeautifulSoup(r.content.decode('utf-8', 'ignore'), 'html.parser')
      data["name"] = detail.find_all("div", class_="pdetail-name")[0].text.split('\n')[1]
      data["url"] = url
      data["image_link"] = images_link[k]
      x=0
      key=''
      for tr in detail.find_all('td'):
        if(len(tr.get_text()) > 0):
          if(x%2)==0:
            key=tr.get_text()
          else:
            data[key]=tr.get_text()
          x=x+1
      data["price"] = detail.find("div", class_="pdetail-price-box").text
      jsonString = json.dumps(data, ensure_ascii=False)
      f.write(jsonString + "\n")
      print(jsonString)
      k = k + 1
    except:
      pass
    # time.sleep(2)


In [ ]:
import json
import pandas as pd

data_to_csv = []
with open("Media_Mart.txt", mode = "r", encoding="utf8") as f:
  for idx, line in enumerate(f):
    data = json.loads(line)
    data_to_csv.append(data)
print(data_to_csv)

In [ ]:
for x in data_to_csv:
    print(x)
    print(x.keys())
    break

In [ ]:
data_new = []
for pair in data_to_csv:
    dt = {}
    for key, value in pair.items():
        key = str(key).strip().replace('\n','')
        dt[key] = value
    data_new.append(dt)

In [ ]:
pd.DataFrame(data_new).to_csv('mediamart.csv')

In [ ]:
df = pd.read_csv("mediamart.csv", index_col=0)

In [ ]:
df["Thương hiệu"] = df["Thương hiệu"].apply(lambda line : str(line).strip())
df["Mã sản phẩm"] = df["Mã sản phẩm"].apply(lambda line : str(line).strip())
df["price"] = df["price"].apply(lambda line : str(line).strip()).apply(lambda line: int(line.split("\n")[0].strip()[:-2].replace(".","")))

In [ ]:
df.head()

In [ ]:
for column, value in df.isnull().sum().sort_values(ascending=True).items():
    print(column, value)

In [ ]:
df = df.loc[:, df.isnull().sum() < 144]

In [ ]:
df.isnull().sum().sort_values(ascending=True)

In [ ]:
df.to_csv("mediamart.csv", index=0)

In [ ]:
df_new = pd.read_csv("mediamart.csv")

In [ ]:
df_new["Màn hình"]